#### SQLite port_lite database: orders table
#### csv files: orders, ORDERS-PLACED

In [1]:
import calendar
import pandas as pd
import panel as pn
import seaborn as sns
from datetime import date, timedelta
from sqlalchemy import create_engine
from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=False)

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

def categorise(row):  
    if row['trade'] == 'B':
        return 'Buy'
    else:
        return 'Sell'
    
today = date.today()
today

<IPython.core.display.Javascript object>

datetime.date(2022, 2, 19)

### Begin of Tables in the process

In [42]:
cols = 'trade name qty price active reason market'.split()
colt = 'trans name qty target active spd current change percent reason market'.split()

In [3]:
format_dict = {
    'qty':'{:,}',
    'price':'{:.2f}','target':'{:.2f}','current':'{:.2f}','change':'{:.2f}','diff':'{:.2f}',
    'amount':'{:,.2f}','sell_amt':'{:,.2f}'
}

In [4]:
pd.read_sql_query('SELECT * FROM orders ORDER BY id DESC LIMIT 1', conlite).style.format(format_dict)

,id,trade,name,qty,price,active,reason,market
0,36,S,WHART,"10,000",12.80,2,DOS,SET


In [5]:
tmp = pd.read_csv('../data/orders-log.csv')
tmp['trans'] = tmp.apply(lambda row: categorise(row), axis=1)
tmp['amount'] = tmp.qty * tmp.target
tmp[tmp.active == 2].style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount


In [6]:
tmp.head().sort_values('spd',ascending=False).style.hide_index().format(format_dict)

trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
B,MAKRO,-3,3L,SET,"3,000",42.00,42.75,1.00,+2.40%,0,Buy,"126,000.00"
B,BGC,-4,DOS,SET,"10,000",10.00,10.40,-0.10,-0.95%,0,Buy,"100,000.00"
B,ASP,-5,6L,SET,"30,000",3.72,3.82,0.00,0.00%,0,Buy,"111,600.00"
B,KKP,-8,3L,SET100,"3,000",68.00,70.00,-1.00,-1.41%,0,Buy,"204,000.00"
B,ASK,-10,1L,SET,"3,000",42.00,44.50,-0.25,-0.56%,0,Buy,"126,000.00"


### End of Tables in the process

### Print to verify before upload file

In [20]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)

df_tab = pn.widgets.Tabulator(orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=   trade    name  ..., width=800)

In [21]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of transactional process

In [22]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE active = 2
ORDER BY trade, name'''
df_act_2 = pd.read_sql(sql, conlite)
df_act_2.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,S,GLOBAL,"7,500",21.00,"157,500.00",5pct,SET50,2
1,S,RATCH,"3,000",46.00,"138,000.00",CP1S,SET50,2
2,S,STA,"2,500",30.25,"75,625.00",CP3S,SET100,2
3,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [23]:
orders_grp = df_act_2.groupby(['trade'])
orders_grp['amount'].sum()

trade
S    499125.0
Name: amount, dtype: float64

In [24]:
mask = df_act_2.trade == 'B'
buy_stocks = df_act_2[mask]
buy_stocks.style.background_gradient(cmap = 'Blues').format(format_dict)

,trade,name,qty,price,amount,reason,market,active


In [25]:
cash = 399_000
buy_amt = buy_stocks.amount.sum()
balance = cash - buy_amt 
cash, buy_amt, balance

(417000, 0.0, 417000.0)

In [26]:
mask = df_act_2.trade == 'S'
sell_stocks = df_act_2[mask]
sell_stocks.style.format(format_dict)

,trade,name,qty,price,amount,reason,market,active
0,S,GLOBAL,"7,500",21.00,"157,500.00",5pct,SET50,2
1,S,RATCH,"3,000",46.00,"138,000.00",CP1S,SET50,2
2,S,STA,"2,500",30.25,"75,625.00",CP3S,SET100,2
3,S,WHART,"10,000",12.80,"128,000.00",DOS,SET,2


In [27]:
sell_amt = round(sell_stocks.amount.sum(),2)
balance = cash + sell_amt 
(cash, sell_amt, balance)

(417000, 499125.0, 916125.0)

### After call ord-log (must call everytime that orders change)

In [28]:
df = pd.read_csv('../data/orders-log.csv')
df['trans'] = df.apply(lambda row: categorise(row), axis=1)
df['amount'] = df.qty * df.target

In [29]:
show(df[['trans','name','spd','reason','qty','target','amount','current','active']])

trans,name,spd,reason,qty,target,amount,current,active


In [30]:
mb = (df.trans == 'Buy') & (df.spd >= -3)
df[mb].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
4,B,MAKRO,-3,3L,SET,"3,000",42.00,42.75,1.00,+2.40%,1,Buy,"126,000.00"
7,B,PTTGC,-3,RD15pct,SET50,"1,200",54.25,55.00,-1.25,-2.22%,1,Buy,"65,100.00"
13,B,TMT,-3,ROUND,SET,"15,000",10.10,10.40,0.00,0.00%,1,Buy,"151,500.00"
14,B,TQM,-2,3L,SET100,"4,500",45.00,45.50,0.00,0.00%,1,Buy,"202,500.00"


In [32]:
show(df[['trans','name','spd','reason','qty','target','current','percent','active']])

trans,name,spd,reason,qty,target,current,percent,active


In [33]:
ms = (df.trans == 'Sell') & (df.spd <= 3)
df[ms].sort_values(['name'],ascending=[True]).style.format(format_dict)

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
16,S,CKP,1,5pct,SET100,"10,000",5.25,5.20,0.10,+1.96%,1,Sell,"52,500.00"
18,S,GLOBAL,2,5pct,SET50,"7,500",21.00,20.80,0.10,+0.48%,2,Sell,"157,500.00"
25,S,RATCH,0,CP1S,SET50,"3,000",46.00,46.00,-0.50,-1.08%,2,Sell,"138,000.00"
27,S,RJH,3,CP1S,SET,"1,500",33.25,32.50,0.00,0.00%,1,Sell,"49,875.00"
30,S,STA,0,CP3S,SET100,"2,500",30.25,30.25,-1.75,-5.47%,2,Sell,"75,625.00"
35,S,WHART,3,DOS,SET,"10,000",12.80,12.50,0.10,+0.81%,2,Sell,"128,000.00"


In [34]:
df[mb | ms].sort_values(['trade','name'],ascending=[True,True])

,trade,name,spd,reason,market,qty,target,current,change,percent,active,trans,amount
4,B,MAKRO,-3,3L,SET,3000,42.00,42.75,1.00,+2.40%,1,Buy,126000.0
7,B,PTTGC,-3,RD15pct,SET50,1200,54.25,55.00,-1.25,-2.22%,1,Buy,65100.0
13,B,TMT,-3,ROUND,SET,15000,10.10,10.40,0.00,0.00%,1,Buy,151500.0
14,B,TQM,-2,3L,SET100,4500,45.00,45.50,0.00,0.00%,1,Buy,202500.0
16,S,CKP,1,5pct,SET100,10000,5.25,5.20,0.10,+1.96%,1,Sell,52500.0
18,S,GLOBAL,2,5pct,SET50,7500,21.00,20.80,0.10,+0.48%,2,Sell,157500.0
25,S,RATCH,0,CP1S,SET50,3000,46.00,46.00,-0.50,-1.08%,2,Sell,138000.0
27,S,RJH,3,CP1S,SET,1500,33.25,32.50,0.00,0.00%,1,Sell,49875.0
30,S,STA,0,CP3S,SET100,2500,30.25,30.25,-1.75,-5.47%,2,Sell,75625.0
35,S,WHART,3,DOS,SET,10000,12.80,12.50,0.10,+0.81%,2,Sell,128000.0


In [35]:
df[mb | ms].shape[0]

10

In [36]:
mask = (df.trade == 'B')
df[mb | ms].nlargest(5, "amount")[["name","trans","amount"]].style.format(format_dict)

,name,trans,amount
14,TQM,Buy,"202,500.00"
18,GLOBAL,Sell,"157,500.00"
13,TMT,Buy,"151,500.00"
25,RATCH,Sell,"138,000.00"
35,WHART,Sell,"128,000.00"


### Select source of orders between these two

In [43]:
df_out = df[mb | ms][colt]
df_out.style.format(format_dict)

,trans,name,qty,target,active,spd,current,change,percent,reason,market
4,Buy,MAKRO,"3,000",42.00,1,-3,42.75,1.00,+2.40%,3L,SET
7,Buy,PTTGC,"1,200",54.25,1,-3,55.00,-1.25,-2.22%,RD15pct,SET50
13,Buy,TMT,"15,000",10.10,1,-3,10.40,0.00,0.00%,ROUND,SET
14,Buy,TQM,"4,500",45.00,1,-2,45.50,0.00,0.00%,3L,SET100
16,Sell,CKP,"10,000",5.25,1,1,5.20,0.10,+1.96%,5pct,SET100
18,Sell,GLOBAL,"7,500",21.00,2,2,20.80,0.10,+0.48%,5pct,SET50
25,Sell,RATCH,"3,000",46.00,2,0,46.00,-0.50,-1.08%,CP1S,SET50
27,Sell,RJH,"1,500",33.25,1,3,32.50,0.00,0.00%,CP1S,SET
30,Sell,STA,"2,500",30.25,2,0,30.25,-1.75,-5.47%,CP3S,SET100
35,Sell,WHART,"10,000",12.80,2,3,12.50,0.10,+0.81%,DOS,SET


In [ ]:
df_out = df[df.active == 2][colt]
df_out.style.format(format_dict)

In [44]:
file_name = 'ORDERS-PLACED.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_out.sort_values(['trans','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)

### End of Output ORDERS-PLACED

In [ ]:
sql = '''
SELECT trade, name, qty, price, qty * price AS amount, reason, market, active
FROM orders
WHERE ACTIVE = 2
ORDER BY trade, name'''
orders = pd.read_sql(sql, conlite)
orders.style.format(format_dict)

### Evening Process

In [ ]:
new_active = 1
old_active = 2

In [ ]:
sql = """
UPDATE orders 
SET active = %s
WHERE active = %s
"""
sql = sql % (new_active, old_active)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
names = df_out['name']
in_p = ", ".join(map(lambda name: "'%s'" % name, names))
in_p

In [ ]:
sql = """
UPDATE orders 
SET active = 2
WHERE name IN (%s)
"""
sql = sql % (in_p)
rp = conlite.execute(sql)
rp.rowcount

In [ ]:
#To force orders.csv and orders table to be same copy
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

orders[cols].to_csv(output_file, header=True, index=False)
orders[cols].to_csv(data_file, header=True, index=False)
orders[cols].to_csv(box_file, header=True, index=False)
orders[cols].to_csv(one_file, header=True, index=False)

### End of Evening Process